In [1]:
!pip install pymssql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 5.2 MB/s 


In [2]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import QuantileTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from joblib import dump

In [3]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" # paste password between quote marks
database = "ghz"

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

In [4]:

data = pd.read_sql(
    """
    select ticker, date, ret, roeq, mom12m
    from data
    where date='2021-12'
    """, 
    conn
)
data = data.dropna()
data['rnk'] = data.ret.rank(pct=True)

In [21]:
transform1 = QuantileTransformer(
    output_distribution="normal"
)

transform2 = QuantileTransformer(
    output_distribution="normal"
)

poly = PolynomialFeatures(degree=2)
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(
   random_state=0
)

pipe = make_pipeline(
  transform1, 
  poly,
  transform2,
  model
)

In [22]:
param_grid = {
    "gradientboostingregressor__max_depth": [3, 4, 5],
    "gradientboostingregressor__learning_rate": [.1, .5, 1],
    "gradientboostingregressor__loss": ["squared_error", "absolute_error"]
}

cv = GridSearchCV(
  pipe, 
  param_grid=param_grid
)

X = data[["roeq", "mom12m"]]
y = data["rnk"]

cv.fit(X, y)

dump(cv, "forest2.joblib")

['forest2.joblib']

In [23]:
cv.best_params_

{'gradientboostingregressor__learning_rate': 0.1,
 'gradientboostingregressor__loss': 'squared_error',
 'gradientboostingregressor__max_depth': 3}

In [24]:
cv.score(X, y)

0.26527142561180195

In [14]:
from joblib import load
Bob = load("forest2.joblib")
Bob.score(X,y)

0.14015535638090215

In [26]:
cv.cv_results_

{'mean_fit_time': array([0.41625295, 0.50277705, 0.5912992 , 0.81600561, 0.8398828 ,
        1.06638222, 0.40420198, 0.50097837, 0.59803691, 0.61828375,
        0.7809864 , 0.9711587 , 0.40752597, 0.50069971, 0.59654069,
        0.61117926, 0.77014403, 0.90895452]),
 'std_fit_time': array([0.01724794, 0.00806109, 0.0022335 , 0.12794194, 0.02035337,
        0.04178157, 0.00636395, 0.00708156, 0.00498134, 0.01633095,
        0.00587443, 0.02440218, 0.00492012, 0.00607468, 0.00337913,
        0.01236095, 0.0170866 , 0.02277246]),
 'mean_score_time': array([0.01017208, 0.0087935 , 0.00884647, 0.01175318, 0.00843039,
        0.00916176, 0.00864396, 0.0085629 , 0.0087832 , 0.00844269,
        0.00836916, 0.00883031, 0.0086235 , 0.00856991, 0.01030798,
        0.00839143, 0.00864363, 0.00838823]),
 'std_score_time': array([0.00250268, 0.00065441, 0.00034137, 0.00285169, 0.00017973,
        0.00074255, 0.00031957, 0.00020442, 0.00024918, 0.0003849 ,
        0.00030333, 0.00061777, 0.00029177, 